# นาย ธภัทร มีชัยธนา รหัส 6402539

# สร้างตัวแปรแล้วทำการ Connect server service ของ MongoDB ที่ทำการ import มาจาก pymongo
# แล้วทำการสร้างตัวแปรที่ใส่ชื่อฐานข้อมูลแล้วเก็บไว้ในตัวแปร และ ใช้ตัวแปรที่เก็บชื่อฐานข้อมูลมากำหนดชื่อของ collection แล้วเก็บในอีกตัวแปรนึง

In [3]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
filter={}

mydb = client['6402539_Final']
mycol = mydb['Gun_Deaths_In_America']


In [5]:
import pandas as pd
import json